In [1]:
%autoreload 2

In [2]:
import random

from IPython.core.display import display, HTML
from tqdm.notebook import tnrange
from mutation_experiments import *
import ast_printer

In [3]:
args = parser.parse_args([])

grammar = open(args.grammar_file).read()
grammar_parser = tatsu.compile(grammar) 

asts = load_asts(args, grammar_parser)

In [4]:
NUM_GAMES = 100

regrwoth_mutated_asts = []

for seed in tnrange(NUM_GAMES):
    validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
    sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)
    random.seed(seed)
    try:
        mutated_ast = sampler.sample_regrowth()
    except SamplingFailedException:
        continue
    regrwoth_mutated_asts.append((mutated_ast, seed))
        


In [83]:
NUM_GAMES = 100
START = 100
single_step_mutated_asts = []

for seed in tnrange(NUM_GAMES):
    validators = [ASTVariableValidator(), ASTScoringPreferenceValidator()]
    sampler = ASTRegrowthSampler(grammar_parser, asts, mutation_prob=0.5, validators=validators)
    random.seed(START + seed)
    try:
        mutated_ast = sampler.sample_single_step()
    except:
        continue
    single_step_mutated_asts.append((mutated_ast, START + seed))
        


In [6]:
def new_ast_to_buffer(new_asts, index):
    mutated_ast, seed = new_asts[index]
    ast_printer.reset_buffers(True)
    ast_printer.pretty_print_ast(mutated_ast, context=dict(html=True))


def visualize(new_asts, index):
    mutated_ast, seed = new_asts[index]
    print(f'With random seed {seed}, mutated:\r\n')
    new_ast_to_buffer(new_asts, index)
    display(HTML('\r\n'.join(ast_printer.BUFFER)))


# Games generated with full regrwoth

I'll pring out some interesting games, and try to explain the game and the change that was made. 

The color-coding on these is perhaps not the most informative, but:
* <span style="color: #AA00FF;">Purple:</span> the original node substituted in
* <span style="color: #FF6A00;">Orange:</span> any nodes sampled under the original node from another game
* <span style="color: #00FF80;">Green:</span> nodes from the substituted subtree (the purple one) where we replaced at least one value under the node, but not the entire node.

**TL;DR: if it's colored, it was't in the original game**


In the game below, it changed the quantification and type from `?g - game_object` to `?g - dodgeball`.

Originally, that clause was marking keeping the beachball (`?b`) in the air for 20 counts -- by providing it's not or touching any other object.

Now it stipulates the beachball can't touch or be in the dodgeball, which is rather more trivial


In [14]:
visualize(regrwoth_mutated_asts, 0)

With random seed 0, mutated:



This is another similar change. Originally, this quantified `?b - bed` like in the previous game, to mark that the player should make the throw from the bed. 

With this change, it marks that the player should make the throw while standing on a block, which is cute.

In [15]:
visualize(regrwoth_mutated_asts, 2)

With random seed 2, mutated:



Funny failure mode -- regardless of what this attempted to convey originally, this can never occur, obviously.

In [16]:
visualize(regrwoth_mutated_asts, 4)

With random seed 4, mutated:



Another amusing failure mode -- ths mutation performed substituted in the predicate `(on ?1 ?2)`, and renaming variables to match the local context created an identical predicate to the one that's already there. 

In [17]:
visualize(regrwoth_mutated_asts, 9)

With random seed 9, mutated:



Changes the scoring from awarding five points for each succesful throw (`throwLandsInTarget`) to five points for each throw. Makes the game easier?

In [18]:
visualize(regrwoth_mutated_asts, 11)

With random seed 11, mutated:



Another amusing failure mode that occurs a bunch -- a predicate being called with the wrong number of arguments as result of a mutation.

Semantically, I considered `(agent_holds ?1)` to accept one argument, and `(on ?1 ?2)` to accept two, and here they ended up flipped.

In [19]:
visualize(regrwoth_mutated_asts, 15)

With random seed 15, mutated:



This game originally was something like "consturct a a tunnel from the flat and bridge blocks, then throw the ball under the bridge.

The change essentially makes it such that you have to construct a tunnel from the ramp and bridge blocks, but it doesn't enforce the first setup term, which was designed to make sure this bridge is constructed on the floor.

In [29]:
visualize(regrwoth_mutated_asts, 25)

With random seed 26, mutated:



Changes the terminal conditions, which were originally `chairStoppedRotating` happening once (that is, the chair stops rotating), to `objectLandsOnRotatingChair` happening once, that is, ending at the first succesful shot. 

Not nonsense, but doesn't make too much sense either.

In [32]:
visualize(regrwoth_mutated_asts, 27)

With random seed 28, mutated:



Changes this game from balancing on the large triangular ramp (what it was before) to balancing on a cube block, which sounds like it would be rather harder?

In [43]:
visualize(regrwoth_mutated_asts, 40)

With random seed 41, mutated:



Game changed from bowling a dodgeball to bowling a cube block

In [45]:
visualize(regrwoth_mutated_asts, 42)

With random seed 43, mutated:



Another minor semantic change, from bouncing a ball off the wall and catching it, to bouncing a ball off the curved wooden ramp and catching it.

In [64]:
visualize(regrwoth_mutated_asts, 53)

With random seed 55, mutated:



# Games generated by making a single step

In these the color scheme is simpler -- the mutation is in blue (first color in a equential colormap, assuming I might want to eventually mark each mutation, or something)

Changed the setup slightly -- before it was `(game-conserved (= (distance ?b ?r) 2))`, that is, the ramp is 2 meters from the bed, and now it's that the ramp is on the bed. This works.

In [89]:
visualize(single_step_mutated_asts, 3)

With random seed 3, mutated:



Another minor semantic difference -- from ending the game after three roll attempts, to ending it after three succesful rolls.

In [93]:
visualize(single_step_mutated_asts, 6)

With random seed 6, mutated:



I think the other substitution method also did something like this -- though it bounced it off the curved ramp, and this bounces it off the hexagonal bin

In [115]:
visualize(single_step_mutated_asts, 19)

With random seed 21, mutated:



Throw the cellphone such that it is adjacent to the doggie bed, rather than such that it lands on it. Cute and valid.m

In [117]:
visualize(single_step_mutated_asts, 21)

With random seed 23, mutated:



Originally, that preference asked to to the chair with the pillow. Now it asks that after throwing the pillow, and before the pillow stops moving (or the agent picks it up again), the chair will move. Presumably this could happen because the agent moves it, or the pillow hits it. Totally valid.

In [121]:
visualize(single_step_mutated_asts, 24)

With random seed 26, mutated:



I'm mildly stunned this worked, but it worked -- this is what was there before: `(exists (?b2 - (either cube_block tall_cylindrical_block short_cylindrical_block)) (on ?b1 ?b2))`


Now it asks if there's another game object, that is in the building, and `?b1` is on the game object `?o2` -- it basically changes the game from "build a tower with the blocks" to "build a tower, starting from a block, and using any object you want, but you only get points for each block you use" -- so you could use other objects to stabilize, for example.

In [137]:
visualize(single_step_mutated_asts, 38)

With random seed 45, mutated:



# Debugging

### TODO:
* Handle `either_types` printing, scoring arithmetic expressions, cases where only the variable list is new.
* Do all of this by changing the high-level handling of mutations to a decorator in the printing function.

In [123]:
import mutation_experiments
mutation_experiments._find_mutations(single_step_mutated_asts[25])

{'type_names': ['baseketball', 'dodgeball', 'beachball'], 'parseinfo': [None, 'either_types', 723, 763, 0, 0], 'mutation': 0}


In [50]:
new_ast_to_buffer(regrwoth_mutated_asts, 45)
[line for line in ast_printer.BUFFER if 'color' in line]

[]

In [27]:
for idx, ast in enumerate(mutated_asts):
    new_ast_to_buffer(mutated_asts, idx)
    if sum(['color' in line for line in ast_printer.BUFFER]) == 0:
        print(idx)